In [7]:
from google.colab import drive
import pandas as pd
import numpy as np


In [1]:
from multiprocessing import Pool
import multiprocessing as mp
import time




In [6]:

def worker():
    proc = mp.current_process()
    print(proc.name)
    print(proc.pid)
    print("SubProcess End")

def worker2():
    proc = mp.current_process()
    print(proc.name)
    print(proc.pid)
    print("SubProcess2 End")

if __name__ == "__main__":
    # main process
    proc = mp.current_process()
    print(proc.name)
    print(proc.pid)

    # process spawning
    p = mp.Process(name="SubProcess", target=worker)
    q = mp.Process(name="SubProcess2", target=worker2)
    p.start()
    q.start()

    print("MainProcess End")

MainProcess
219
SubProcessSubProcess2

MainProcess End

1533SubProcess End
1536
SubProcess2 End


In [8]:
drive.mount('/content/drive')
%cd /content/drive/MyDrive/LCB proj

!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/LCB proj
 clashsimulator.py		    LCB_skill_spec.csv
'Clash Win Rate Simulator.ipynb'    LCB_skill_spec.gsheet
 example_sample			    machine_learning.ipynb
 example_sample.csv		    __pycache__
'GoogleColab Excel Convert.ipynb'


In [9]:
import clashsimulator as Clash

In [10]:
skill_informations = pd.read_csv("/content/drive/MyDrive/LCB proj/LCB_skill_spec.csv")

In [11]:
skill_informations

,캐릭터,인격 등급,인격,스킬 이름,스킬 위력,스킬 코인 갯수,코인 위력,비고
0,이상,1.0,LCB 수감자,쳐내기,3,1,7,NaN
1,NaN,NaN,NaN,밀어 찌르기,4,2,4,NaN
2,NaN,NaN,NaN,연격,6,3,1,NaN
3,NaN,2.0,세븐 협회 6과,플레쉬,4,1,6,NaN
4,NaN,NaN,NaN,리포스트,5,2,2,NaN
...,...,...,...,...,...,...,...,...
121,NaN,NaN,NaN,몰아치기,5,3,1,NaN
122,NaN,NaN,NaN,연무 장타,4,4,2,NaN
123,NaN,3.0,G사 일등대리,베어내기,4,2,3,NaN
124,NaN,NaN,NaN,사지절단,6,1,10,NaN


In [12]:
skill_informations.loc[0]

캐릭터              이상
인격 등급           1.0
인격          LCB 수감자
스킬 이름           쳐내기
스킬 위력             3
스킬 코인 갯수          1
코인 위력             7
비고              NaN
Name: 0, dtype: object

In [13]:
skill_informations.isnull().loc[1]['인격']

True

In [14]:
skill_data = []
character = None
personality = None

for i in range(len(skill_informations)):

    if skill_informations.isnull().loc[i]['인격'] == False:
        if personality != None:
            character.append(personality)
        personality = [skill_informations.loc[i]['인격']]
    
    
    if skill_informations.isnull().loc[i]['캐릭터'] == False:
        if character != None:
            #character.append(personality)
            skill_data.append(character)
        character = [skill_informations.loc[i]['캐릭터']]
        
    personality.append(
        [
            skill_informations.loc[i]['스킬 이름'],
            skill_informations.loc[i]['스킬 위력'],
            skill_informations.loc[i]['스킬 코인 갯수'],
            skill_informations.loc[i]['코인 위력']
        ]
    )
character.append(personality)
skill_data.append(character)

In [15]:
skill_data 

[['이상',
  ['LCB 수감자', ['쳐내기', 3, 1, 7], ['밀어 찌르기', 4, 2, 4], ['연격', 6, 3, 1]],
  ['세븐 협회 6과', ['플레쉬', 4, 1, 6], ['리포스트', 5, 2, 2], ['물리네', 5, 3, 3]],
  ['검계 살수', ['격세', 5, 1, 6], ['두의틀기', 7, 2, 1], ['이면찬자', 8, 3, 1]]],
 ['파우스트',
  ['LCB 수감자', ['내려베기', 3, 1, 7], ['올려베기', 4, 2, 4], ['후벼찌르기', 6, 2, 2]],
  ['W사 정리 요원', ['에너지순환', 4, 1, 6], ['도약', 5, 2, 4], ['과충전', 6, 3, 2]],
  ['살아남은 로보토미 직원', ['단격', 3, 1, 6], ['깊게베기', 4, 3, 3], ['기회노리기', 4, 2, 8]],
  ['쥐는 자', ['광소', 4, 2, 3], ['쥐어짐', 4, 3, 4], ['처단', 6, 3, 2]]],
 ['돈키호테',
  ['LCB 수감자', ['찌르기', 3, 1, 7], ['모아 찌르기', 4, 1, 10], ['정의를 위해!', 3, 3, 3]],
  ['시 협회 5과', ['숨고르기', 4, 1, 6], ['이격', 4, 2, 6], ['과호흡', 4, 1, 21]],
  ['W사 정리 요원', ['절단', 5, 1, 6], ['도약', 4, 3, 4], ['공간 절단', 1, 5, 2]]],
 ['로슈',
  ['LCB 수감자', ['칠하기', 3, 1, 7], ['흩뿌리기', 4, 2, 4], ['그려내기', 5, 3, 2]],
  ['세븐 협회 6과', ['슬래시', 4, 1, 6], ['어퍼 슬래시', 4, 1, 12], ['스워시', 4, 3, 5]],
  ['흑운회 와카슈', ['발경', 5, 1, 6], ['뒷처리', 5, 2, 5], ['구름 휘젓기', 8, 2, 4]]],
 ['뫼르소',
  ['LCB 수감자', ['원, 투', 2

In [16]:
learn_data = pd.DataFrame(columns = ['skill force', 'coin count', 'coin force', 'coin probability', 
                                     'skill force_foreign', 'coin count_foreign', 'coin force_foreign', 'coin probability_foreign',
                                     'win rate'])

In [17]:
learn_data_temp = pd.DataFrame(columns = ['skill force', 'coin count', 'coin force', 'coin probability', 
                                     'skill force_foreign', 'coin count_foreign', 'coin force_foreign', 'coin probability_foreign',
                                     'win rate'])

In [18]:
learn_data

,skill force,coin count,coin force,coin probability,skill force_foreign,coin count_foreign,coin force_foreign,coin probability_foreign,win rate


In [19]:
def get_skill(character, personality, skill):
    return character[personality][skill] [1:]


skill1 = get_skill(skill_data[0],1,3)

skill2 = get_skill(skill_data[1], 1, 2)
print(skill1, "\n", skill2)

[6, 3, 1] 
 [4, 2, 4]


In [20]:
result = Clash.get_skill_prob_list(skill1, skill2, 2, 10)

result

{'x': array([0.09090909, 0.18181818, 0.27272727, 0.36363636, 0.45454545,
        0.54545455, 0.63636364, 0.72727273, 0.81818182, 0.90909091]),
 'y': array([0.09090909, 0.18181818, 0.27272727, 0.36363636, 0.45454545,
        0.54545455, 0.63636364, 0.72727273, 0.81818182, 0.90909091]),
 'z': [[1.0, 1.0, 0.5, 0.5, 1.0, 0.5, 1.0, 0.0, 0.0, 0.0],
  [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [1.0, 1.0, 1.0, 1.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0],
  [1.0, 0.5, 1.0, 0.0, 0.0, 1.0, 0.0, 0.5, 0.0, 0.0],
  [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.5, 0.0],
  [1.0, 1.0, 1.0, 1.0, 0.5, 0.5, 0.5, 0.0, 0.0, 0.0],
  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0],
  [1.0, 0.5, 1.0, 1.0, 0.5, 1.0, 1.0, 0.5, 0.0, 0.0],
  [1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0.0, 0.0, 0.0, 0.0]]}

In [21]:
extract_quantity = 300
clash_repeat = 30
mentality_repeat = 100

In [22]:
def data_gen():
    for i in range(1):
        for q in range(1, 2): #임시 data extration용 
    #for i in range(len(skill_data)-1):
    #    for q in range(i+1, len(skill_data)):    
            global learn_data_temp

            character1 = skill_data[i][1:]
            character2 = skill_data[q][1:]
            character1_name = skill_data[i][0]
            character2_name = skill_data[q][0]

            for personality1 in character1:
                for personality2 in character2:
                    person1 = personality1[1:]
                    person2 = personality2[1:]

                    for skillA in person1:
                        for skillB in person2:
                            print(personality1[0], skillA, personality2[0], skillB)
                            skillforceA = []
                            coincountA = []
                            coinforceA = []
                            coinprobabilityA = []
                            skillforceB = []
                            coincountB = []
                            coinforceB = []
                            coinprobabilityB = []
                            winrate = []

                            Clash_result = Clash.get_skill_prob_list(skillA[1:], skillB[1:], clash_repeat, mentality_repeat)
                            for cnt in range(extract_quantity):
                                x_ = int(np.random.uniform(mentality_repeat))
                                y_ = int(np.random.uniform(mentality_repeat))
                                x = Clash_result['x'][x_]
                                y = Clash_result['y'][y_]
                                z = Clash_result['z'][x_][y_]

                                skillforceA.append(skillA[1])
                                coincountA.append(skillA[2])
                                coinforceA.append(skillA[3])
                                coinprobabilityA.append(x)
                                skillforceB.append(skillB[1])
                                coincountB.append(skillB[2])
                                coinforceB.append(skillB[3])
                                coinprobabilityB.append(y)
                                winrate.append(z)


                            data_temp = {
                                'skill force'                 : skillforceA,
                                'coin count'                  : coincountA,
                                'coin force'                  : coinforceA,
                                'coin probability'            : coinprobabilityA,
                                'skill force_foreign'         : skillforceB,
                                'coin count_foreign'          : coincountB,
                                'coin force_foreign'          : coinforceB,
                                'coin probability_foreign'    : coinprobabilityB,
                                'win rate'                    : winrate
                            }

                            learn_data_temp = pd.concat([learn_data_temp, pd.DataFrame(data_temp)])
                            
            print(character1_name + " : " + character2_name +"  ::  data extract completed.")

In [ ]:
data_gen()

LCB 수감자 ['쳐내기', 3, 1, 7] LCB 수감자 ['내려베기', 3, 1, 7]
LCB 수감자 ['쳐내기', 3, 1, 7] LCB 수감자 ['올려베기', 4, 2, 4]
LCB 수감자 ['쳐내기', 3, 1, 7] LCB 수감자 ['후벼찌르기', 6, 2, 2]
LCB 수감자 ['밀어 찌르기', 4, 2, 4] LCB 수감자 ['내려베기', 3, 1, 7]
LCB 수감자 ['밀어 찌르기', 4, 2, 4] LCB 수감자 ['올려베기', 4, 2, 4]
LCB 수감자 ['밀어 찌르기', 4, 2, 4] LCB 수감자 ['후벼찌르기', 6, 2, 2]
LCB 수감자 ['연격', 6, 3, 1] LCB 수감자 ['내려베기', 3, 1, 7]
LCB 수감자 ['연격', 6, 3, 1] LCB 수감자 ['올려베기', 4, 2, 4]
LCB 수감자 ['연격', 6, 3, 1] LCB 수감자 ['후벼찌르기', 6, 2, 2]
LCB 수감자 ['쳐내기', 3, 1, 7] W사 정리 요원 ['에너지순환', 4, 1, 6]
LCB 수감자 ['쳐내기', 3, 1, 7] W사 정리 요원 ['도약', 5, 2, 4]
LCB 수감자 ['쳐내기', 3, 1, 7] W사 정리 요원 ['과충전', 6, 3, 2]
LCB 수감자 ['밀어 찌르기', 4, 2, 4] W사 정리 요원 ['에너지순환', 4, 1, 6]
LCB 수감자 ['밀어 찌르기', 4, 2, 4] W사 정리 요원 ['도약', 5, 2, 4]
LCB 수감자 ['밀어 찌르기', 4, 2, 4] W사 정리 요원 ['과충전', 6, 3, 2]
LCB 수감자 ['연격', 6, 3, 1] W사 정리 요원 ['에너지순환', 4, 1, 6]
LCB 수감자 ['연격', 6, 3, 1] W사 정리 요원 ['도약', 5, 2, 4]
LCB 수감자 ['연격', 6, 3, 1] W사 정리 요원 ['과충전', 6, 3, 2]
LCB 수감자 ['쳐내기', 3, 1, 7] 살아남은 로보토미 직원 ['단격', 3, 1, 6]
LCB 수감자 ['

In [ ]:
learn_data_temp

,skill force,coin count,coin force,coin probability,skill force_foreign,coin count_foreign,coin force_foreign,coin probability_foreign,win rate
0,3,1,7,0.356436,3,1,7,0.237624,0.700000
1,3,1,7,0.118812,3,1,7,0.584158,0.100000
2,3,1,7,0.039604,3,1,7,0.514851,0.000000
3,3,1,7,0.752475,3,1,7,0.108911,1.000000
4,3,1,7,0.683168,3,1,7,0.792079,0.400000
...,...,...,...,...,...,...,...,...,...
295,8,3,1,0.851485,6,3,2,0.059406,1.000000
296,8,3,1,0.930693,6,3,2,0.485149,1.000000
297,8,3,1,0.603960,6,3,2,0.485149,0.933333
298,8,3,1,0.782178,6,3,2,0.742574,0.600000


In [ ]:
learn_data_temp.to_csv("example_sample.csv")